In [4]:
# 安装Python库rpy2
!pip install rpy2

# 安装R包INLA及其依赖包
!R -e "install.packages(c('sp', 'fmesher'), repos=c('http://cran.r-project.org'))"
!R -e "install.packages('INLA', repos='https://inla.r-inla-download.org/R/stable')"


R version 4.3.2 (2023-10-31) -- "Eye Holes"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> install.packages(c('sp', 'fmesher'), repos=c('http://cran.r-project.org'))
Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘s2’, ‘units’, ‘sf’, ‘Rcpp’

trying URL 'http://cran.r-project.org/src/contrib/proxy_0

In [5]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# 自动转换Pandas DataFrame为R的data.frame
pandas2ri.activate()

# 加载R的INLA库
inla = importr('INLA')
base = importr('base')

In [7]:
import pandas as pd

# 加载数据
data_path = "/content/TestData_California.csv"
df = pd.read_csv(data_path)


In [14]:
import pandas as pd
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.rinterface_lib.embedded import RRuntimeError

# 激活Pandas DataFrame到R data.frame的自动转换
pandas2ri.activate()

# 尝试加载INLA库，捕获任何错误
try:
    inla = ro.r('library(INLA)')
except RRuntimeError as e:
    print(f"Failed to load INLA library in R: {e}")

def run_inla_model(formula, data):
    """
    使用INLA运行贝叶斯层次模型。
    :param formula: R风格的模型公式字符串。
    :param data: 数据集，Pandas DataFrame格式。
    """
    try:
        # 将Pandas DataFrame转换为R的data.frame
        r_df = pandas2ri.py2rpy(data)

        # 设置R全局环境中的数据框
        ro.globalenv['r_df'] = r_df

        # 运行INLA模型
        model_fit = ro.r(f"""
        inla(formula = '{formula}', data = r_df, family = 'gaussian',
             control.predictor = list(compute = TRUE))
        """)

        # 打印模型摘要
        print(ro.r('summary')(model_fit))
    except RRuntimeError as e:
        print(f"Error running INLA model: {e}")

# 加载数据
df = pd.read_csv('/content/TestData_California.csv')

# 定义模型公式
model_formulas = {
    "Model 1": "SentimentScore ~ HeatCount"
}

# 运行模型
for model_name, formula in model_formulas.items():
    print(f"Running {model_name} with formula: {formula}")
    run_inla_model(formula, df)
    print("\n" + "="*50 + "\n")


Running Model 1 with formula: SentimentScore ~ HeatCount


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():



 *** inla.core.safe:  The inla program failed, but will rerun in case better initial values may help. try=1/1 


  object 'y...fake' not found
The inla program failed and the maximum number of tries has been reached.



Error running INLA model: Error in inla.core.safe(formula = formula, family = family, contrasts = contrasts,  : 
  object 'y...fake' not found
The inla program failed and the maximum number of tries has been reached.





In [15]:
import pandas as pd
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# 激活Pandas DataFrame到R data.frame的自动转换
pandas2ri.activate()

# 加载R的INLA库
ro.r('library(INLA)')

def run_inla_model(formula, data):
    """
    使用INLA运行贝叶斯层次模型。
    :param formula: R风格的模型公式字符串。
    :param data: 数据集，Pandas DataFrame格式。
    """
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_df = ro.conversion.py2rpy(data)

    # 构建R中的INLA模型调用字符串
    inla_call = f"""
    result <- inla(formula = {formula}, data = r_df, family = 'gaussian',
                   control.predictor = list(compute = TRUE))
    """

    # 在R环境中执行INLA模型调用
    ro.r(inla_call)

    # 获取模型结果并打印摘要
    ro.r('print(summary(result))')

# 加载你的数据
df = pd.read_csv('/content/TestData_California.csv')  # 确保路径与你的文件匹配

# 定义模型公式
model_formulas = {
    "Model 1": "SentimentScore ~ HeatCount"
}

# 运行模型
for model_name, formula in model_formulas.items():
    print(f"Running {model_name} with formula: {formula}")
    run_inla_model(formula, df)
    print("\n" + "="*50 + "\n")


Running Model 1 with formula: SentimentScore ~ HeatCount


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():



Call:
   c("inla.core(formula = formula, family = family, contrasts = contrasts, 
   ", " data = data, quantiles = quantiles, E = E, offset = offset, ", " 
   scale = scale, weights = weights, Ntrials = Ntrials, strata = strata, 
   ", " lp.scale = lp.scale, link.covariates = link.covariates, verbose = 
   verbose, ", " lincomb = lincomb, selection = selection, control.compute 
   = control.compute, ", " control.predictor = control.predictor, 
   control.family = control.family, ", " control.inla = control.inla, 
   control.fixed = control.fixed, ", " control.mode = control.mode, 
   control.expert = control.expert, ", " control.hazard = control.hazard, 
   control.lincomb = control.lincomb, ", " control.update = 
   control.update, control.lp.scale = control.lp.scale, ", " 
   control.pardiso = control.pardiso, only.hyperparam = only.hyperparam, 
   ", " inla.call = inla.call, inla.arg = inla.arg, num.threads = 
   num.threads, ", " keep = keep, working.directory = working.directory,

In [24]:
import pandas as pd
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# 激活Pandas DataFrame到R data.frame的自动转换
pandas2ri.activate()

# 加载R的INLA库
ro.r('library(INLA)')

def run_inla_model(formula, data):
    """
    使用INLA运行贝叶斯层次模型。
    :param formula: R风格的模型公式字符串。
    :param data: 数据集，Pandas DataFrame格式。
    """
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_df = ro.conversion.py2rpy(data)

    # 构建R中的INLA模型调用字符串
    inla_call = f"""
    result <- inla(formula = {formula}, data = r_df, family = 'gaussian',
                   control.predictor = list(compute = TRUE))
    """

    # 在R环境中执行INLA模型调用
    ro.r(inla_call)

    # 获取模型结果并打印摘要
    ro.r('print(summary(result))')


# 加载你的数据
df = pd.read_csv('/content/TestData_California.csv')  # 确保路径与你的文件匹配


# 定义模型公式
model_formulas = {
    "Model 1": "SentimentScore ~ HeatCount",
    "Model 2": "SentimentScore ~ AirPolllution_Interpolate",
    "Model 3": "SentimentScore ~ HeatCount + AirPolllution_Interpolate",
    "Model 4": "SentimentScore ~ HeatCount + AirPolllution_Interpolate + Holiday + Year + Month + Week + Weekend",
    "Model 5": "SentimentScore ~ HeatCount + AirPolllution_Interpolate + Holiday + Year + Month + Week + Weekend + Humidity + Precipitation + Solar + VulnerabilityIndex",
    # 对于Model 6，我们需要先筛选出2020和2021年的数据
}

# 注意：对于Model 6的实现，你需要在运行模型前筛选出2020和2021年的数据
df_2020_2021 = df[df['Year'].isin([2020, 2021])]

model_formulas["Model 6"] = "SentimentScore ~ HeatCount + AirPolllution_Interpolate + Holiday + Year + Month + Week + Weekend + Humidity + Precipitation + Solar + VulnerabilityIndex + StringencyIndex"

# 运行模型
for model_name, formula in model_formulas.items():
    if model_name == "Model 6":
        print(f"Running {model_name} with formula: {formula}")
        run_inla_model(formula, df_2020_2021)
    else:
        print(f"Running {model_name} with formula: {formula}")
        run_inla_model(formula, df)
    print("\n" + "="*50 + "\n")





Running Model 1 with formula: SentimentScore ~ HeatCount


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():



Call:
   c("inla.core(formula = formula, family = family, contrasts = contrasts, 
   ", " data = data, quantiles = quantiles, E = E, offset = offset, ", " 
   scale = scale, weights = weights, Ntrials = Ntrials, strata = strata, 
   ", " lp.scale = lp.scale, link.covariates = link.covariates, verbose = 
   verbose, ", " lincomb = lincomb, selection = selection, control.compute 
   = control.compute, ", " control.predictor = control.predictor, 
   control.family = control.family, ", " control.inla = control.inla, 
   control.fixed = control.fixed, ", " control.mode = control.mode, 
   control.expert = control.expert, ", " control.hazard = control.hazard, 
   control.lincomb = control.lincomb, ", " control.update = 
   control.update, control.lp.scale = control.lp.scale, ", " 
   control.pardiso = control.pardiso, only.hyperparam = only.hyperparam, 
   ", " inla.call = inla.call, inla.arg = inla.arg, num.threads = 
   num.threads, ", " keep = keep, working.directory = working.directory,